In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
import gc
import scipy as sp
import os
import sys
import lightgbm as lgb
from sklearn.metrics import mean_absolute_error
import scipy as sp
from sklearn.model_selection import KFold
from scipy.signal import hilbert
from scipy.signal import hann
from scipy.signal import convolve
from sklearn.model_selection import train_test_split
from sklearn import datasets
from sklearn import metrics
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2,f_regression
import matplotlib.pyplot as plt

In [2]:
# Definitions
data_dir = "/home/slow-storage/Kaggle/Earthquake/"
train_file = data_dir + "train.csv"

test_dir = data_dir + "/test/"

In [3]:
traindf = pd.read_csv(train_file,
                      dtype={
                             'acoustic_data': np.int16, 
                             'time_to_failure': np.float64
                      },
                     )

In [4]:
rows = 150000
chunk = int(np.floor(traindf.shape[0] / rows))

x_traindf = pd.DataFrame(index=range(chunk), dtype=np.float64,
                       columns=['mean', 'std', 'var', 'max', 'min'])
y_traindf = pd.DataFrame(index=range(chunk), dtype=np.float64,
                       columns=['time_to_failure'])

In [21]:
def augment_data(df,x,index):
   
    first_segments = [10000,50000]
    last_segments  = [10000,50000]
    mid_segments   = [30000,50000]
    
    quantiles=[0.01,0.05,0.10,0.15,0.20,0.25,0.30,0.35,0.40,0.45,0.50,
               0.55,0.60,0.65,0.70,0.75,0.80,0.85,0.90,0.85,0.90,0.95,0.99]
    
    z = np.fft.fft(x)
    realFFT = np.real(z)
    imagFFT = np.imag(z)
    
    for f_seg in first_segments:
        first_set = x[:f_seg]
        df.loc[index, 'std_first_{}'.format(f_seg)] = x[:f_seg].std()
        df.loc[index, 'mean_first_{}'.format(f_seg)] = x[:f_seg].mean()
        df.loc[index, 'Rstd_first_{}'.format(f_seg)] = realFFT[:f_seg].std()
        df.loc[index, 'Rmean_first_{}'.format(f_seg)] = realFFT[:f_seg].mean()
        df.loc[index, 'mean_first_{}'.format(f_seg)] = first_set.mean()
        df.loc[index, 'std_first_{}'.format(f_seg)] = first_set.std()
        df.loc[index, 'var_first_{}'.format(f_seg)] = first_set.var()
        df.loc[index, 'max_first_{}'.format(f_seg)] = first_set.max()
        df.loc[index, 'min_first_{}'.format(f_seg)] = first_set.min()
        df.loc[index, 'sum_first_{}'.format(f_seg)] = first_set.sum()
        df.loc[index, 'ptp_first_{}'.format(f_seg)] = np.ptp(first_set)
        df.loc[index, 'abs_max_first_{}'.format(f_seg)] = np.abs(first_set).max()
        df.loc[index, 'abs_min_first_{}'.format(f_seg)] = np.abs(first_set).min()
        df.loc[index, 'mean_change_abs_first_{}'.format(f_seg)] = np.mean(np.diff(first_set))
        df.loc[index, 'abs_mean_first_{}'.format(f_seg)] = np.abs(first_set).mean()
        df.loc[index, 'abs_std_first_{}'.format(f_seg)] = np.abs(first_set).std()
        df.loc[index, 'Hilbert_mean_first_{}'.format(f_seg)] = np.abs(hilbert(first_set)).mean()
        df.loc[index, 'max_to_min_first_{}'.format(f_seg)] = first_set.max() / np.abs(first_set.min())
        df.loc[index, 'max_to_min_diff_first_{}'.format(f_seg)] = first_set.max() - np.abs(first_set.min())
        df.loc[index, 'skew_first_{}'.format(f_seg)]      = sp.stats.skew(first_set)
        df.loc[index, 'kurtosis_first_{}'.format(f_seg)]  = sp.stats.kurtosis(first_set)
        df.loc[index, 'entropy_first_{}'.format(f_seg)]   = sp.stats.entropy(first_set)
        df.loc[index, 'sem_first_{}'.format(f_seg)]       = sp.stats.sem(first_set, axis=None, ddof=0)
        df.loc[index, 'trim_mean_first_{}'.format(f_seg)] = sp.stats.trim_mean(first_set, 0.1)
        df.loc[index, 'variation_first_{}'.format(f_seg)] = sp.stats.variation(first_set)
        df.loc[index, 'gmean_first_{}'.format(f_seg)]     = sp.stats.gmean(first_set)
        df.loc[index, 'tmean_first_{}'.format(f_seg)]     = sp.stats.tmean(first_set)
        df.loc[index, 'tstd_first_{}'.format(f_seg)]      = sp.stats.tstd(first_set)
        df.loc[index, 'tsem_first_{}'.format(f_seg)]      = sp.stats.tsem(first_set)
        df.loc[index, 'kstat_1_first_{}'.format(f_seg)]   = sp.stats.kstat(first_set, 1)
        df.loc[index, 'kstat_2_first_{}'.format(f_seg)]   = sp.stats.kstat(first_set, 2)
        df.loc[index, 'kstat_3_first_{}'.format(f_seg)]   = sp.stats.kstat(first_set, 3)
        df.loc[index, 'kstat_4_first_{}'.format(f_seg)]   = sp.stats.kstat(first_set, 4)
        df.loc[index, 'moment_1_first_{}'.format(f_seg)]  = sp.stats.moment(first_set, 1)
        df.loc[index, 'moment_2_first_{}'.format(f_seg)]  = sp.stats.moment(first_set, 2)
        df.loc[index, 'moment_3_first_{}'.format(f_seg)]  = sp.stats.moment(first_set, 3)
        df.loc[index, 'moment_4_first_{}'.format(f_seg)]  = sp.stats.moment(first_set, 4)
    
        for n in quantiles:
            df.loc[index, 'abs_q_first_{}_{}'.format(n,f_seg)] = np.quantile(np.abs(first_set), n)
    
        for n in quantiles:
            df.loc[index, 'q_first_{}_{}'.format(n,f_seg)] = np.quantile(first_set, n)
            
            
    for l_seg in last_segments:
        last_set = x[-l_seg:]
        df.loc[index, 'std_last_{}'.format(l_seg)] = last_set.std()
        df.loc[index, 'mean_last_{}'.format(l_seg)] = last_set.mean()
        df.loc[index, 'Rstd_last_{}'.format(l_seg)] = realFFT[-l_seg:].std()
        df.loc[index, 'Rmean_last_{}'.format(l_seg)] = realFFT[-l_seg:].mean()
        df.loc[index, 'mean_last_{}'.format(l_seg)] = last_set.mean()
        df.loc[index, 'std_last_{}'.format(l_seg)] = last_set.std()
        df.loc[index, 'var_last_{}'.format(l_seg)] = last_set.var()
        df.loc[index, 'max_last_{}'.format(l_seg)] = last_set.max()
        df.loc[index, 'min_last_{}'.format(l_seg)] = last_set.min()
        df.loc[index, 'sum_last_{}'.format(l_seg)] = last_set.sum()
        df.loc[index, 'ptp_last_{}'.format(l_seg)] = np.ptp(last_set)
        df.loc[index, 'abs_max_last_{}'.format(l_seg)] = np.abs(last_set).max()
        df.loc[index, 'abs_min_last_{}'.format(l_seg)] = np.abs(last_set).min()
        df.loc[index, 'mean_change_abs_last_{}'.format(l_seg)] = np.mean(np.diff(last_set))
        df.loc[index, 'abs_mean_last_{}'.format(l_seg)] = np.abs(last_set).mean()
        df.loc[index, 'abs_std_last_{}'.format(l_seg)] = np.abs(last_set).std()
        df.loc[index, 'Hilbert_mean_last_{}'.format(l_seg)] = np.abs(hilbert(last_set)).mean()
        df.loc[index, 'max_to_min_last_{}'.format(l_seg)] = last_set.max() / np.abs(last_set.min())
        df.loc[index, 'max_to_min_diff_last_{}'.format(l_seg)] = last_set.max() - np.abs(last_set.min())
        df.loc[index, 'skew_last_{}'.format(l_seg)]      = sp.stats.skew(last_set)
        df.loc[index, 'kurtosis_last_{}'.format(l_seg)]  = sp.stats.kurtosis(last_set)
        df.loc[index, 'entropy_last_{}'.format(l_seg)]   = sp.stats.entropy(last_set)
        df.loc[index, 'sem_last_{}'.format(l_seg)]       = sp.stats.sem(last_set, axis=None, ddof=0)
        df.loc[index, 'trim_mean_last_{}'.format(l_seg)] = sp.stats.trim_mean(last_set, 0.1)
        df.loc[index, 'variation_last_{}'.format(l_seg)] = sp.stats.variation(last_set)
        df.loc[index, 'gmean_last_{}'.format(l_seg)]     = sp.stats.gmean(last_set)
        df.loc[index, 'tmean_last_{}'.format(l_seg)]     = sp.stats.tmean(last_set)
        df.loc[index, 'tstd_last_{}'.format(l_seg)]      = sp.stats.tstd(last_set)
        df.loc[index, 'tsem_last_{}'.format(l_seg)]      = sp.stats.tsem(last_set)
        df.loc[index, 'kstat_1_last_{}'.format(l_seg)]   = sp.stats.kstat(last_set, 1)
        df.loc[index, 'kstat_2_last_{}'.format(l_seg)]   = sp.stats.kstat(last_set, 2)
        df.loc[index, 'kstat_3_last_{}'.format(l_seg)]   = sp.stats.kstat(last_set, 3)
        df.loc[index, 'kstat_4_last_{}'.format(l_seg)]   = sp.stats.kstat(last_set, 4)
        df.loc[index, 'moment_1_last_{}'.format(l_seg)]  = sp.stats.moment(last_set, 1)
        df.loc[index, 'moment_2_last_{}'.format(l_seg)]  = sp.stats.moment(last_set, 2)
        df.loc[index, 'moment_3_last_{}'.format(l_seg)]  = sp.stats.moment(last_set, 3)
        df.loc[index, 'moment_4_last_{}'.format(l_seg)]  = sp.stats.moment(last_set, 4)
    
        for n in quantiles:
            df.loc[index, 'abs_q_last_{}_{}'.format(n,l_seg)] = np.quantile(np.abs(last_set), n)
    
        for n in quantiles:
            df.loc[index, 'q_last_{}_{}'.format(n,l_seg)] = np.quantile(last_set, n)
    
    for m_seg in mid_segments:
        mid_set = x[m_seg:-m_seg]
        df.loc[index, 'std_mid_{}'.format(m_seg)] = mid_set.std()
        df.loc[index, 'mean_mid_{}'.format(m_seg)] = mid_set.mean()
        df.loc[index, 'Rstd_mid_{}'.format(m_seg)] = realFFT[-m_seg:m_seg].std()
        df.loc[index, 'Rmean_mid_{}'.format(m_seg)] = realFFT[-m_seg:m_seg].mean()     
        df.loc[index, 'mean_mid_{}'.format(m_seg)] = mid_set.mean()
        df.loc[index, 'std_mid_{}'.format(m_seg)] = mid_set.std()
        df.loc[index, 'var_mid_{}'.format(m_seg)] = mid_set.var()
        df.loc[index, 'max_mid_{}'.format(m_seg)] = mid_set.max()
        df.loc[index, 'min_mid_{}'.format(m_seg)] = mid_set.min()
        df.loc[index, 'sum_mid_{}'.format(m_seg)] = mid_set.sum()
        df.loc[index, 'ptp_mid_{}'.format(m_seg)] = np.ptp(mid_set)
        df.loc[index, 'abs_max_mid_{}'.format(m_seg)] = np.abs(mid_set).max()
        df.loc[index, 'abs_min_mid_{}'.format(m_seg)] = np.abs(mid_set).min()
        df.loc[index, 'mean_change_abs_mid_{}'.format(m_seg)] = np.mean(np.diff(mid_set))
        df.loc[index, 'abs_mean_mid_{}'.format(m_seg)] = np.abs(mid_set).mean()
        df.loc[index, 'abs_std_mid_{}'.format(m_seg)] = np.abs(mid_set).std()
        df.loc[index, 'Hilbert_mean_mid_{}'.format(m_seg)] = np.abs(hilbert(mid_set)).mean()
        df.loc[index, 'max_to_min_mid_{}'.format(m_seg)] = mid_set.max() / np.abs(mid_set.min())
        df.loc[index, 'max_to_min_diff_mid_{}'.format(m_seg)] = mid_set.max() - np.abs(mid_set.min())
        df.loc[index, 'skew_mid_{}'.format(m_seg)]      = sp.stats.skew(mid_set)
        df.loc[index, 'kurtosis_mid_{}'.format(m_seg)]  = sp.stats.kurtosis(mid_set)
        df.loc[index, 'entropy_mid_{}'.format(m_seg)]   = sp.stats.entropy(mid_set)
        df.loc[index, 'sem_mid_{}'.format(m_seg)]       = sp.stats.sem(mid_set, axis=None, ddof=0)
        df.loc[index, 'trim_mean_mid_{}'.format(m_seg)] = sp.stats.trim_mean(mid_set, 0.1)
        df.loc[index, 'variation_mid_{}'.format(m_seg)] = sp.stats.variation(mid_set)
        df.loc[index, 'gmean_mid_{}'.format(m_seg)]     = sp.stats.gmean(mid_set)
        df.loc[index, 'tmean_mid_{}'.format(m_seg)]     = sp.stats.tmean(mid_set)
        df.loc[index, 'tstd_mid_{}'.format(m_seg)]      = sp.stats.tstd(mid_set)
        df.loc[index, 'tsem_mid_{}'.format(m_seg)]      = sp.stats.tsem(mid_set)
        df.loc[index, 'kstat_1_mid_{}'.format(m_seg)]   = sp.stats.kstat(mid_set, 1)
        df.loc[index, 'kstat_2_mid_{}'.format(m_seg)]   = sp.stats.kstat(mid_set, 2)
        df.loc[index, 'kstat_3_mid_{}'.format(m_seg)]   = sp.stats.kstat(mid_set, 3)
        df.loc[index, 'kstat_4_mid_{}'.format(m_seg)]   = sp.stats.kstat(mid_set, 4)
        df.loc[index, 'moment_1_mid_{}'.format(m_seg)]  = sp.stats.moment(mid_set, 1)
        df.loc[index, 'moment_2_mid_{}'.format(m_seg)]  = sp.stats.moment(mid_set, 2)
        df.loc[index, 'moment_3_mid_{}'.format(m_seg)]  = sp.stats.moment(mid_set, 3)
        df.loc[index, 'moment_4_mid_{}'.format(m_seg)]  = sp.stats.moment(mid_set, 4)
    
        for n in quantiles:
            df.loc[index, 'abs_q_mid_{}_{}'.format(n,m_seg)] = np.quantile(np.abs(mid_set), n)
    
        for n in quantiles:
            df.loc[index, 'q_mid_{}_{}'.format(n,m_seg)] = np.quantile(mid_set, n)

    #Get Statistics on Whole segment
    df.loc[index, 'Real_mean'] = realFFT.mean()
    df.loc[index, 'Real_std'] = realFFT.std()
    df.loc[index, 'Imag_mean'] = imagFFT.mean()
    df.loc[index, 'Imag_std'] = imagFFT.std()
    df.loc[index, 'mean'] = x.mean()
    df.loc[index, 'std'] = x.std()
    df.loc[index, 'var'] = x.var()
    df.loc[index, 'max'] = x.max()
    df.loc[index, 'min'] = x.min()
    df.loc[index, 'sum'] = x.sum()
    df.loc[index, 'ptp'] = np.ptp(x)
    df.loc[index, 'abs_max'] = np.abs(x).max()
    df.loc[index, 'abs_min'] = np.abs(x).min()
    df.loc[index, 'mean_change_abs'] = np.mean(np.diff(x))
    df.loc[index, 'abs_mean'] = np.abs(x).mean()
    df.loc[index, 'abs_std'] = np.abs(x).std()
    df.loc[index, 'Hilbert_mean'] = np.abs(hilbert(x)).mean()
    df.loc[index, 'max_to_min'] = x.max() / np.abs(x.min())
    df.loc[index, 'max_to_min_diff'] = x.max() - np.abs(x.min())
    df.loc[index, 'skew']      = sp.stats.skew(x)
    df.loc[index, 'kurtosis']  = sp.stats.kurtosis(x)
    df.loc[index, 'entropy']   = sp.stats.entropy(x)
    df.loc[index, 'sem']       = sp.stats.sem(x, axis=None, ddof=0)
    df.loc[index, 'trim_mean'] = sp.stats.trim_mean(x, 0.1)
    df.loc[index, 'variation'] = sp.stats.variation(x)
    df.loc[index, 'gmean']     = sp.stats.gmean(x)
    df.loc[index, 'tmean']     = sp.stats.tmean(x)
    df.loc[index, 'tstd']      = sp.stats.tstd(x)
    df.loc[index, 'tsem']      = sp.stats.tsem(x)
    df.loc[index, 'kstat_1']   = sp.stats.kstat(x, 1)
    df.loc[index, 'kstat_2']   = sp.stats.kstat(x, 2)
    df.loc[index, 'kstat_3']   = sp.stats.kstat(x, 3)
    df.loc[index, 'kstat_4']   = sp.stats.kstat(x, 4)
    df.loc[index, 'moment_1']  = sp.stats.moment(x, 1)
    df.loc[index, 'moment_2']  = sp.stats.moment(x, 2)
    df.loc[index, 'moment_3']  = sp.stats.moment(x, 3)
    df.loc[index, 'moment_4']  = sp.stats.moment(x, 4)
    
    for n in quantiles:
        df.loc[index, 'abs_q{}'.format(n)] = np.quantile(np.abs(x), n)
    
    for n in quantiles:
        df.loc[index, 'q{}'.format(n)] = np.quantile(x, n)



In [22]:
total_samples = 629145480
slide_ruler=10000

totalloops=int(total_samples/slide_ruler)
print(totalloops)
for item in range(totalloops):
    print(item*slide_ruler,item*slide_ruler+rows)
    section = traindf.iloc[item*slide_ruler:item*slide_ruler+rows]
    x = section['acoustic_data'].values
    y = section['time_to_failure'].values[-1]
    y_traindf.loc[item, 'time_to_failure'] = y
    
    augment_data(x_traindf,x,item)
    


62914
0 150000


/home/slow-storage/local/Anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:112: RuntimeWarning: Mean of empty slice.


10000 160000
20000 170000
30000 180000
40000 190000
50000 200000
60000 210000
70000 220000
80000 230000
90000 240000
100000 250000
110000 260000
120000 270000
130000 280000
140000 290000
150000 300000
160000 310000
170000 320000
180000 330000
190000 340000
200000 350000
210000 360000
220000 370000
230000 380000
240000 390000
250000 400000
260000 410000
270000 420000
280000 430000
290000 440000
300000 450000
310000 460000
320000 470000
330000 480000
340000 490000
350000 500000
360000 510000
370000 520000
380000 530000
390000 540000
400000 550000
410000 560000
420000 570000
430000 580000
440000 590000
450000 600000
460000 610000
470000 620000
480000 630000
490000 640000
500000 650000
510000 660000
520000 670000
530000 680000
540000 690000
550000 700000
560000 710000
570000 720000
580000 730000
590000 740000
600000 750000
610000 760000
620000 770000
630000 780000
640000 790000
650000 800000
660000 810000
670000 820000
680000 830000
690000 840000
700000 850000
710000 860000
720000 870000
7

5260000 5410000
5270000 5420000
5280000 5430000
5290000 5440000
5300000 5450000
5310000 5460000
5320000 5470000
5330000 5480000
5340000 5490000
5350000 5500000
5360000 5510000
5370000 5520000
5380000 5530000
5390000 5540000
5400000 5550000
5410000 5560000
5420000 5570000
5430000 5580000
5440000 5590000
5450000 5600000
5460000 5610000
5470000 5620000
5480000 5630000
5490000 5640000
5500000 5650000
5510000 5660000
5520000 5670000
5530000 5680000
5540000 5690000
5550000 5700000
5560000 5710000
5570000 5720000
5580000 5730000
5590000 5740000
5600000 5750000
5610000 5760000
5620000 5770000
5630000 5780000
5640000 5790000
5650000 5800000
5660000 5810000
5670000 5820000
5680000 5830000
5690000 5840000
5700000 5850000
5710000 5860000
5720000 5870000
5730000 5880000
5740000 5890000
5750000 5900000
5760000 5910000
5770000 5920000
5780000 5930000
5790000 5940000
5800000 5950000
5810000 5960000
5820000 5970000
5830000 5980000
5840000 5990000
5850000 6000000
5860000 6010000
5870000 6020000
5880000 

10330000 10480000
10340000 10490000
10350000 10500000
10360000 10510000
10370000 10520000
10380000 10530000
10390000 10540000
10400000 10550000
10410000 10560000
10420000 10570000
10430000 10580000
10440000 10590000
10450000 10600000
10460000 10610000
10470000 10620000
10480000 10630000
10490000 10640000
10500000 10650000
10510000 10660000
10520000 10670000
10530000 10680000
10540000 10690000
10550000 10700000
10560000 10710000
10570000 10720000
10580000 10730000
10590000 10740000
10600000 10750000
10610000 10760000
10620000 10770000
10630000 10780000
10640000 10790000
10650000 10800000
10660000 10810000
10670000 10820000
10680000 10830000
10690000 10840000
10700000 10850000
10710000 10860000
10720000 10870000
10730000 10880000
10740000 10890000
10750000 10900000
10760000 10910000
10770000 10920000
10780000 10930000
10790000 10940000
10800000 10950000
10810000 10960000
10820000 10970000
10830000 10980000
10840000 10990000
10850000 11000000
10860000 11010000
10870000 11020000
10880000 1

...

617970000 618120000
617980000 618130000
617990000 618140000
618000000 618150000
618010000 618160000
618020000 618170000
618030000 618180000
618040000 618190000
618050000 618200000
618060000 618210000
618070000 618220000
618080000 618230000
618090000 618240000
618100000 618250000
618110000 618260000
618120000 618270000
618130000 618280000
618140000 618290000
618150000 618300000
618160000 618310000
618170000 618320000
618180000 618330000
618190000 618340000
618200000 618350000
618210000 618360000
618220000 618370000
618230000 618380000
618240000 618390000
618250000 618400000
618260000 618410000
618270000 618420000
618280000 618430000
618290000 618440000
618300000 618450000
618310000 618460000
618320000 618470000
618330000 618480000
618340000 618490000
618350000 618500000
618360000 618510000
618370000 618520000
618380000 618530000
618390000 618540000
618400000 618550000
618410000 618560000
618420000 618570000
618430000 618580000
618440000 618590000
618450000 618600000
618460000 618610000


622070000 622220000
622080000 622230000
622090000 622240000
622100000 622250000
622110000 622260000
622120000 622270000
622130000 622280000
622140000 622290000
622150000 622300000
622160000 622310000
622170000 622320000
622180000 622330000
622190000 622340000
622200000 622350000
622210000 622360000
622220000 622370000
622230000 622380000
622240000 622390000
622250000 622400000
622260000 622410000
622270000 622420000
622280000 622430000
622290000 622440000
622300000 622450000
622310000 622460000
622320000 622470000
622330000 622480000
622340000 622490000
622350000 622500000
622360000 622510000
622370000 622520000
622380000 622530000
622390000 622540000
622400000 622550000
622410000 622560000
622420000 622570000
622430000 622580000
622440000 622590000
622450000 622600000
622460000 622610000
622470000 622620000
622480000 622630000
622490000 622640000
622500000 622650000
622510000 622660000
622520000 622670000
622530000 622680000
622540000 622690000
622550000 622700000
622560000 622710000


626170000 626320000
626180000 626330000
626190000 626340000
626200000 626350000
626210000 626360000
626220000 626370000
626230000 626380000
626240000 626390000
626250000 626400000
626260000 626410000
626270000 626420000
626280000 626430000
626290000 626440000
626300000 626450000
626310000 626460000
626320000 626470000
626330000 626480000
626340000 626490000
626350000 626500000
626360000 626510000
626370000 626520000
626380000 626530000
626390000 626540000
626400000 626550000
626410000 626560000
626420000 626570000
626430000 626580000
626440000 626590000
626450000 626600000
626460000 626610000
626470000 626620000
626480000 626630000
626490000 626640000
626500000 626650000
626510000 626660000
626520000 626670000
626530000 626680000
626540000 626690000
626550000 626700000
626560000 626710000
626570000 626720000
626580000 626730000
626590000 626740000
626600000 626750000
626610000 626760000
626620000 626770000
626630000 626780000
626640000 626790000
626650000 626800000
626660000 626810000


/home/slow-storage/local/Anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:110: RuntimeWarning: Mean of empty slice.
/home/slow-storage/local/Anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:113: RuntimeWarning: Mean of empty slice.
/home/slow-storage/local/Anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:115: RuntimeWarning: Degrees of freedom <= 0 for slice


ValueError: zero-size array to reduction operation maximum which has no identity

In [23]:
# write these to a file so we don't have to keep generating them
x_traindf.to_csv(data_dir+'trainx_augment.csv',index=False)
y_traindf.to_csv(data_dir+'trainy_augment.csv',index=False)

In [15]:
x_traindf = pd.read_csv(data_dir+'trainx_augment.csv')
y_traindf = pd.read_csv(data_dir+'trainy_augment.csv')
print(x_traindf.shape,y_traindf.shape)

(62914, 74) (62914, 1)


In [16]:
pd.options.mode.use_inf_as_na = True

# Drop columns with too many NaNs 
x_traindf = x_traindf.dropna(axis='columns',thresh=100)
x_traindf = x_traindf.fillna(0)




In [17]:
x = x_traindf.to_numpy()
y = y_traindf.to_numpy()
y = y.flatten()


print("Number of Samples: ", x.shape[0],"\nNumber of Features: ", x.shape[1])


columns = x_traindf.columns

#del x_traindf
#del x_traindf
#del y_traindf
gc.collect()

Number of Samples:  62914 
Number of Features:  70


725

### Scaling Data

The StandardScaler() function Scikit Learn standardizes the data by removing the mean and unit variance.

In [18]:
scaler = StandardScaler()
scaler.fit(x)
scaled_x = scaler.transform(x)

### Feature Importance

Which of the features created above actually make a difference to the target variable?



In [19]:
### Feature Importance
select = SelectKBest(score_func=f_regression, k=14)
select = select.fit(x_traindf, y_traindf)

Kbestdf = pd.DataFrame(data=select.scores_,index=columns)
# Summarize scores
print(Kbestdf.sort_values(by=0,ascending=False))

                             0
q80               18862.322495
q90               17282.277563
q85               16594.267237
q95               13252.846668
q05               12629.273667
q75               10859.266722
kstat_2           10046.391512
abs_q95            8628.856036
q70                7779.361270
kstat_3            7082.104968
Hilbert_mean       6850.363931
q99                6524.904462
q01                6517.644138
abs_mean           5640.160696
q65                4780.223842
abs_q99            3421.238688
Rstd_last_10000    3349.239688
Rstd_last_50000    3175.224727
Imag_std           3122.702652
std                3121.993673
tstd               3121.993657
sem                3120.414775
tsem               3120.414735
variation          3089.937196
abs_std            2664.546392
min                2466.952362
ptp                2449.500396
max                2330.525925
abs_max            2314.998341
Real_std           2229.113810
...                        ...
min_last

/home/slow-storage/local/Anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/slow-storage/local/Anaconda3/lib/python3.7/site-packages/sklearn/feature_selection/univariate_selection.py:299: RuntimeWarning: invalid value encountered in true_divide
  corr /= X_norms
/home/slow-storage/local/Anaconda3/lib/python3.7/site-packages/scipy/stats/_distn_infrastructure.py:877: RuntimeWarning: invalid value encountered in greater
  return (self.a < x) & (x < self.b)
/home/slow-storage/local/Anaconda3/lib/python3.7/site-packages/scipy/stats/_distn_infrastructure.py:877: RuntimeWarning: invalid value encountered in less
  return (self.a < x) & (x < self.b)
/home/slow-storage/local/Anaconda3/lib/python3.7/site-packages/scipy/stats/_distn_infrastructure.py:1831: RuntimeWarning: invalid value

In [20]:
from sklearn.ensemble import ExtraTreesRegressor
model = ExtraTreesRegressor()
model.fit(x_traindf, y_traindf)
treesdf = pd.DataFrame(data=model.feature_importances_,index=columns)
# Summarize scores
print(treesdf.sort_values(by=0,ascending=False))

/home/slow-storage/local/Anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/home/slow-storage/local/Anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  This is separate from the ipykernel package so we can avoid doing imports until


                         0
q95               0.071026
q01               0.060394
q05               0.049078
abs_q99           0.035221
q80               0.034667
std               0.031253
q99               0.029223
max_to_min        0.028412
q85               0.028373
abs_q95           0.028359
abs_max           0.026769
max_to_min_diff   0.026056
Rstd_last_10000   0.025742
kstat_3           0.024469
q90               0.023938
Rstd_last_50000   0.023398
ptp               0.023141
q70               0.022242
max               0.021688
min               0.021227
skew              0.019627
moment_3          0.019591
kstat_4           0.016381
kurtosis          0.014499
abs_mean          0.013398
min_last_50000    0.012345
moment_4          0.012328
sum               0.012188
kstat_1           0.011842
max_last_50000    0.011291
...                    ...
std_first_50000   0.007948
Real_std          0.007062
mean_first_10000  0.006443
mean_last_10000   0.006286
abs_std           0.006228
k

In [21]:
from sklearn.ensemble import AdaBoostRegressor
regr = AdaBoostRegressor(random_state=0, n_estimators=100)
regr.fit(x_traindf, y_traindf)
regrdf = pd.DataFrame(data=model.feature_importances_,index=columns)
# Summarize scores
print(regrdf.sort_values(by=0,ascending=False))

/home/slow-storage/local/Anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


                         0
q95               0.071026
q01               0.060394
q05               0.049078
abs_q99           0.035221
q80               0.034667
std               0.031253
q99               0.029223
max_to_min        0.028412
q85               0.028373
abs_q95           0.028359
abs_max           0.026769
max_to_min_diff   0.026056
Rstd_last_10000   0.025742
kstat_3           0.024469
q90               0.023938
Rstd_last_50000   0.023398
ptp               0.023141
q70               0.022242
max               0.021688
min               0.021227
skew              0.019627
moment_3          0.019591
kstat_4           0.016381
kurtosis          0.014499
abs_mean          0.013398
min_last_50000    0.012345
moment_4          0.012328
sum               0.012188
kstat_1           0.011842
max_last_50000    0.011291
...                    ...
std_first_50000   0.007948
Real_std          0.007062
mean_first_10000  0.006443
mean_last_10000   0.006286
abs_std           0.006228
k

In [22]:
from sklearn.ensemble import RandomForestRegressor
model = RandomForestRegressor()
model.fit(x_traindf, y_traindf)
rtreesdf = pd.DataFrame(data=model.feature_importances_,index=columns)
# Summarize scores
print(rtreesdf.sort_values(by=0,ascending=False))

/home/slow-storage/local/Anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/home/slow-storage/local/Anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  This is separate from the ipykernel package so we can avoid doing imports until


                         0
abs_q95           0.371741
max_to_min        0.037936
q95               0.036943
kstat_4           0.035520
max_to_min_diff   0.028483
min               0.024492
moment_3          0.023270
skew              0.022877
max               0.020687
ptp               0.020425
abs_max           0.017174
kurtosis          0.016687
mean_first_50000  0.016269
kstat_3           0.014742
mean_last_50000   0.014580
moment_4          0.014452
trim_mean         0.013626
abs_mean          0.013364
q05               0.013007
Hilbert_mean      0.012150
max_last_50000    0.011575
Real_std          0.011507
variation         0.010871
max_first_50000   0.010745
min_first_50000   0.010731
min_last_50000    0.010659
std_last_50000    0.010429
mean_last_10000   0.009896
std_first_50000   0.009554
mean_first_10000  0.008705
...                    ...
std_first_10000   0.005261
q85               0.005218
abs_std           0.004740
mean_change_abs   0.004342
q99               0.003897
m

In [23]:
from sklearn.decomposition import PCA

pca = PCA().fit(x_traindf)

pcadf = pd.DataFrame(data=pca.explained_variance_ratio_,index=columns)
# Summarize scores
print(pcadf.sort_values(by=0,ascending=False))

                             0
mean              1.000000e+00
std               1.676636e-09
var               7.111943e-11
max               2.763108e-11
min               3.470701e-13
sum               4.796084e-15
ptp               2.125407e-15
abs_max           1.268598e-15
abs_min           7.979204e-16
mean_change_abs   4.529444e-16
q95               4.468421e-16
q99               3.522611e-16
q05               3.002433e-16
q01               2.420512e-16
abs_q95           1.830388e-16
abs_q99           5.918491e-17
abs_q05           5.243825e-17
abs_q01           4.498742e-17
abs_mean          1.980941e-17
abs_std           1.573395e-17
q70               1.153450e-17
q75               9.684226e-18
q60               8.949827e-18
q65               7.931194e-18
q85               6.412479e-19
q90               3.109944e-19
q80               2.516547e-19
Real_mean         2.329246e-19
Real_std          1.382522e-19
Imag_mean         9.868575e-20
...                        ...
max_firs

In [24]:
feature_importance_values = np.abs(x_traindf.corrwith(y_traindf.time_to_failure))
print(feature_importance_values.sort_values(ascending=False))

q80                 0.480274
q90                 0.464225
q85                 0.456855
q95                 0.417136
q05                 0.408881
q75                 0.383669
kstat_2             0.371080
abs_q95             0.347296
q70                 0.331733
kstat_3             0.318090
Hilbert_mean        0.313362
q99                 0.306543
q01                 0.306389
abs_mean            0.286837
q65                 0.265739
abs_q99             0.227105
Rstd_last_10000     0.224824
Rstd_last_50000     0.219194
Imag_std            0.217460
std                 0.217436
tstd                0.217436
sem                 0.217384
tsem                0.217384
variation           0.216370
abs_std             0.201575
min                 0.194250
ptp                 0.193588
max                 0.189000
abs_max             0.188392
Real_std            0.184986
                      ...   
min_last_10000      0.133677
max_last_10000      0.132008
kstat_4             0.131749
std_first_1000

### Preparing Train and Test Sets

80% of the data will be used for the training set, and 20% will be used for the testing set.  The data will be shuffled to help prevent overfitting.

In [25]:
x_train, x_test, y_train, y_test = train_test_split(
                scaled_x, 
                y, 
                test_size=0.1, 
                random_state=42,
                shuffle=True)

The test and train datasets can be initialized using the Dataset() function from the LightGBM framework.  This will put the dataset into LightGBM format.  This mandatory in order to use this framework, but it is much more memory efficient than Numpy or Pandas.

In [26]:
train_data = lgb.Dataset(x_train, y_train, free_raw_data=False)
test_data  = lgb.Dataset(x_test,  y_test,  free_raw_data=False, reference=train_data)

## Training Model Using LightGBM



In [36]:
model = lgb.LGBMRegressor(
                        device_type="gpu",
                        gpu_device_id=0,   
                        boosting_type= 'gbdt',
                        objective = 'regression_l1',
                        #objective=huber,
                        n_jobs = 4,
                        silent = True,
                        num_iterations=20000,
                        early_stopping_round=200,
                        metric='mae',
                        max_bin = 63,
                        min_split_gain = 0.1,
                        min_child_weight = 0.1,
                        min_child_samples = 20,
                        bagging_freq=5,
                        #helps Overfitting
                        bagging_fraction=.8,
                        num_leaves=64,
                        max_depth=6,          
                        min_data_in_leaf=79,
                        learning_rate=0.01,
                        bagging_seed=11,
                        reg_alpha=0.1302650970728192,
                        reg_lambda=0.3603427518866501,
                         )

# To view the default model params:
for param in model.get_params().keys():
    print(param,":",model.get_params()[param])

boosting_type : gbdt
class_weight : None
colsample_bytree : 1.0
importance_type : split
learning_rate : 0.01
max_depth : 6
min_child_samples : 20
min_child_weight : 0.1
min_split_gain : 0.1
n_estimators : 100
n_jobs : 4
num_leaves : 64
objective : regression_l1
random_state : None
reg_alpha : 0.1302650970728192
reg_lambda : 0.3603427518866501
silent : True
subsample : 1.0
subsample_for_bin : 200000
subsample_freq : 0
device_type : gpu
gpu_device_id : 0
num_iterations : 20000
early_stopping_round : 200
metric : mae
max_bin : 63
bagging_freq : 5
bagging_fraction : 0.8
min_data_in_leaf : 79
bagging_seed : 11


In [37]:
print('Starting training...')

folds = KFold(n_splits=5, shuffle=True, random_state=11)

for fold_n, (train_index, valid_index) in enumerate(folds.split(x_train)):
    print('Fold', fold_n)
    x_train_f, x_test_f = x_train[train_index], x_train[valid_index]
    y_train_f, y_test_f = y_train[train_index], y_train[valid_index]

    model = model.fit(
                       x_train_f,y_train_f.flatten(),
                       eval_set=[(x_test_f, y_test_f.flatten())],
                       eval_metric='mae',
                       verbose=False
                 )

    print('Starting predicting...')
    # predict
    y_pred = model.predict(x_test_f, num_iteration=model.best_iteration_)
    # eval
    print('The mae of prediction is:', mean_absolute_error(y_test_f.flatten(), y_pred))

Starting training...
Fold 0


/home/slow-storage/local/Anaconda3/lib/python3.7/site-packages/lightgbm/engine.py:118: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
/home/slow-storage/local/Anaconda3/lib/python3.7/site-packages/lightgbm/engine.py:123: UserWarning: Found `early_stopping_round` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


Starting predicting...
The mae of prediction is: 1.5719558622320242
Fold 1


/home/slow-storage/local/Anaconda3/lib/python3.7/site-packages/lightgbm/engine.py:118: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
/home/slow-storage/local/Anaconda3/lib/python3.7/site-packages/lightgbm/engine.py:123: UserWarning: Found `early_stopping_round` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


Starting predicting...
The mae of prediction is: 1.5511806773940977
Fold 2


...

Starting predicting...
The mae of prediction is: 1.575711925802127
Fold 3


/home/slow-storage/local/Anaconda3/lib/python3.7/site-packages/lightgbm/engine.py:118: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
/home/slow-storage/local/Anaconda3/lib/python3.7/site-packages/lightgbm/engine.py:123: UserWarning: Found `early_stopping_round` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


Starting predicting...
The mae of prediction is: 1.5554212488948358
Fold 4


/home/slow-storage/local/Anaconda3/lib/python3.7/site-packages/lightgbm/engine.py:118: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
/home/slow-storage/local/Anaconda3/lib/python3.7/site-packages/lightgbm/engine.py:123: UserWarning: Found `early_stopping_round` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


Starting predicting...
The mae of prediction is: 1.5535319098188063


In [29]:
submission = pd.read_csv(data_dir + 'sample_submission.csv', index_col='seg_id')
x_test = pd.DataFrame(columns=columns, dtype=np.float64, index=submission.index)

for seg_id in x_test.index:
    seg = pd.read_csv(data_dir + 'test/{}.csv'.format(seg_id))
    
    x = seg['acoustic_data'].to_numpy()
    
    augment_data(x_test,x,seg_id)


pd.options.mode.use_inf_as_na = True
# Drop columns with too many NaNs 
x_test = x_test.dropna(axis='columns',thresh=100)
x_test = x_test.fillna(0) 

# Use the same fitted Scaler from above
scaled_test_x = scaler.transform(x_test)
# Use the same selector from above
select_test_x = select.transform(scaled_test_x)

/home/slow-storage/local/Anaconda3/lib/python3.7/site-packages/scipy/stats/stats.py:316: RuntimeWarning: divide by zero encountered in log
  log_a = np.log(a)
/home/slow-storage/local/Anaconda3/lib/python3.7/site-packages/scipy/stats/stats.py:316: RuntimeWarning: invalid value encountered in log
  log_a = np.log(a)


In [30]:
print(select_test_x.shape)

(2624, 14)


In [38]:
y_pred=model.predict(scaled_test_x)

In [39]:
submission['time_to_failure'] = y_pred
submission.to_csv(data_dir+'submission.csv')
submission.head()

,time_to_failure
seg_id,
seg_00030f,7.798083
seg_0012b5,5.686530
seg_00184e,6.510664
seg_003339,8.083363
seg_0042cc,7.867685
